# Credits to

[1] https://www.kaggle.com/mayer79/m5-forecast-keras-with-categorical-embeddings-v2 

[2] https://www.kaggle.com/ragnar123/very-fst-model

[3] https://www.kaggle.com/mayer79/m5-forecast-poisson-loss

# Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
import time
import pickle
import lightgbm as lgb

In [2]:
'''
Got from: [1]
'''
path = "../input/m5-forecasting-accuracy"

sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sales = pd.read_csv(os.path.join(path, "sales_train_validation.csv"))

In [3]:
sales2 = sales.drop(["item_id","dept_id","cat_id","store_id","state_id","id"],axis=1)

In [4]:
sales3 = np.transpose(np.array(sales2))

In [5]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

Using TensorFlow backend.


In [6]:
n_steps_in, n_steps_out = 7, 7
for stores in range(10):
    sales4 = sales3[:,3049*stores:3049*stores+3049]
    X, y = split_sequences(sales4, n_steps_in, n_steps_out)
    n_features = X.shape[2]
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=50, verbose=1)
    
    x_input = sales4[-n_steps_in:,:]
    x_input = x_input.reshape((1, n_steps_in, n_features))
    yhat = model.predict(x_input, verbose=0)
    sample_submission.iloc[0:3049,1:8] = np.transpose(yhat[0])
    for i in range(1,4):
        yhat = model.predict(yhat, verbose=0)
        sample_submission.iloc[3049*stores:3049*stores+3049,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])
    for i in range(4):
        yhat = model.predict(yhat, verbose=0)
        sample_submission.iloc[30490+3049*stores:30490+3049+3049*stores,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])
    

Epoch 1/50
1900/1900 [==============================] - 7s 4ms/step - loss: 11.3878
Epoch 2/50
1900/1900 [==============================] - 6s 3ms/step - loss: 7.5730
Epoch 3/50
1900/1900 [==============================] - 6s 3ms/step - loss: 6.9657
Epoch 4/50
1900/1900 [==============================] - 7s 4ms/step - loss: 6.5739
Epoch 5/50
1900/1900 [==============================] - 6s 3ms/step - loss: 6.2311
Epoch 6/50
1900/1900 [==============================] - 6s 3ms/step - loss: 6.0296
Epoch 7/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.8535
Epoch 8/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.7612
Epoch 9/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.6508
Epoch 10/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.5703
Epoch 11/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.5050
Epoch 12/50
1900/1900 [==============================] - 6s 3ms/step - loss: 5.4713


In [7]:
sample_submission.to_csv("submission.csv", index=False)

In [8]:
'''
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# choose a number of time steps
n_steps_in, n_steps_out = 7, 7
# covert into input/output
X, y = split_sequences(sales4, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1, verbose=1)
'''

"\nfrom numpy import array\nfrom numpy import hstack\nfrom keras.models import Sequential\nfrom keras.layers import LSTM\nfrom keras.layers import Dense\nfrom keras.layers import RepeatVector\nfrom keras.layers import TimeDistributed\n \n# split a multivariate sequence into samples\ndef split_sequences(sequences, n_steps_in, n_steps_out):\n\tX, y = list(), list()\n\tfor i in range(len(sequences)):\n\t\t# find the end of this pattern\n\t\tend_ix = i + n_steps_in\n\t\tout_end_ix = end_ix + n_steps_out\n\t\t# check if we are beyond the dataset\n\t\tif out_end_ix > len(sequences):\n\t\t\tbreak\n\t\t# gather input and output parts of the pattern\n\t\tseq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]\n\t\tX.append(seq_x)\n\t\ty.append(seq_y)\n\treturn array(X), array(y)\n \n# choose a number of time steps\nn_steps_in, n_steps_out = 7, 7\n# covert into input/output\nX, y = split_sequences(sales4, n_steps_in, n_steps_out)\n# the dataset knows the number of features, e.g. 2

In [9]:
'''
x_input = sales4[-n_steps_in:,:]
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
sample_submission.iloc[0:3049,1:8] = np.transpose(yhat[0])
for i in range(1,4):
    yhat = model.predict(yhat, verbose=0)
    sample_submission.iloc[0:3049,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])
for i in range(4):
    yhat = model.predict(yhat, verbose=0)
    sample_submission.iloc[0+30490:0+30490+3049,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])
'''

'\nx_input = sales4[-n_steps_in:,:]\nx_input = x_input.reshape((1, n_steps_in, n_features))\nyhat = model.predict(x_input, verbose=0)\nsample_submission.iloc[0:3049,1:8] = np.transpose(yhat[0])\nfor i in range(1,4):\n    yhat = model.predict(yhat, verbose=0)\n    sample_submission.iloc[0:3049,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])\nfor i in range(4):\n    yhat = model.predict(yhat, verbose=0)\n    sample_submission.iloc[0+30490:0+30490+3049,1+n_steps_out*i:8+n_steps_out*i] = np.transpose(yhat[0])\n'